In [1]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

import CMINE_lib as CMINE
# from Guassian_variables import Data_guassian

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [2]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
from modules.CMI import DR_CMI, CDL_CMI

In [5]:
Dim = 5
batch_size = 64
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = CMINE.create_dataset_DGP_binary_A( Dim=5, N=batch_size)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

In [6]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [ ]:
def train_dr(N = 64):
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []

In [7]:
model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
dr_est_values = []

In [8]:
cdl_est_values = []


In [9]:
for step in range(training_steps):
    #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
    dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=64)
    s_t = torch.from_numpy(dataset[0]).float().cuda()
    s_next = torch.from_numpy(dataset[1]).float().cuda()
    a = torch.from_numpy(dataset[2]).float().cuda()
    
    batch_x = torch.cat([s_t,a], dim=1)
    batch_y = s_next
    model_dr.eval()
    drs = model_dr(batch_x, batch_y)
    #mi_est_values.append(cmi)
    dr_est_values.append(drs)
    model_dr.train() 

    model_loss = model_dr.learning_loss(batch_x, batch_y)

    optimizer_dr.zero_grad()
    model_loss.backward(retain_graph=True)
    optimizer_dr.step()

    del batch_x, batch_y
    torch.cuda.empty_cache()

/home/defucao/anaconda3/envs/diffscm_gpu/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [11]:
model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)

for step in range(training_steps):
    #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
    dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=64)
    s_t = torch.from_numpy(dataset[0]).float().cuda()
    s_next = torch.from_numpy(dataset[1]).float().cuda()
    a = torch.from_numpy(dataset[2]).float().cuda()
    
    batch_x = torch.cat([s_t,a], dim=1)
    batch_y = s_next
    model_cdl.eval()
    cdl_cmi = model_cdl(batch_x, batch_y)
    cdl_est_values.append(cdl_cmi)
    model_cdl.train() 

    model_loss = model_cdl.learning_loss(batch_x, batch_y)

    optimizer_cdl.zero_grad()
    model_loss.backward(retain_graph=True)
    optimizer_cdl.step()

    del batch_x, batch_y
    torch.cuda.empty_cache()

In [12]:
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[5.29158881e+01 4.35538159e+03 2.15365230e+00 1.33740542e+00
 7.05669273e+00 4.81182321e+00 1.43800545e+02 6.67770227e-01
 1.96548440e+01 7.68899346e+00]
[8.90897569e+02 8.43244921e+02 8.41170084e+02 9.16700953e+02
 8.10734532e+02 1.91285104e+04 9.28880367e+05 3.69319064e+04
 4.21118277e+07 1.34269863e+05]
